# Reaction Times

![](https://img.shields.io/badge/status-WIP-orange)

## The Data

Data from @wagenmakers2008diffusion - Experiment 1.
Using the same procedure as the authors, we excluded all trials with uninterpretable response time [see @theriault2024check], i.e., responses that are too fast response (<180 ms) or too slow (>2 sec instead of >3 sec). 


In [ ]:
#| code-fold: false

using Downloads, CSV, DataFrames
using Turing, Distributions, SequentialSamplingModels
using CairoMakie

df = CSV.read(Downloads.download("https://raw.githubusercontent.com/DominiqueMakowski/CognitiveModels/main/data/wagenmakers2008.csv"), DataFrame)
first(df, 10)

We create a new column, `Accuracy`, which is the "binarization" of the `Condition` column, and is equal to 1 when the condition is `"Accuracy"` and 0 when it is `"Speed"`.


In [ ]:
#| output: false

df = df[df.Error .== 0, :]
df.Accuracy = df.Condition .== "Accuracy"

::: {.callout-tip title="Code Tip"}
Note the usage of *vectorization* `.==` as we want to compare each element of the `Condition` vector to the target `"Accuracy"`.
:::


In [ ]:
function plot_distribution(df, title="Empirical Distribution of Data from Wagenmakers et al. (2018)")
    fig = Figure()
    ax = Axis(fig[1, 1], title=title,
        xlabel="RT (s)",
        ylabel="Distribution",
        yticksvisible=false,
        xticksvisible=false,
        yticklabelsvisible=false)
    CairoMakie.density!(df[df.Condition .== "Speed", :RT], color=("#EF5350", 0.7), label = "Speed")
    CairoMakie.density!(df[df.Condition .== "Accuracy", :RT], color=("#66BB6A", 0.7), label = "Accuracy")
    CairoMakie.axislegend("Condition"; position=:rt)
    CairoMakie.ylims!(ax, (0, nothing))
    return fig
end

plot_distribution(df, "Empirical Distribution of Data from Wagenmakers et al. (2018)")

## Descriptive Models 

### Bayesian Linear Models

#### The Model


In [ ]:
#| code-fold: false

@model function model_linear(rt; condition=nothing)

    # Set priors on variance, intercept and effect of ISI
    σ ~ truncated(Normal(0, 1); lower=0)
    intercept ~ truncated(Normal(0, 1); lower=0)
    slope_accuracy ~ Normal(0, 0.5)

    for i in 1:length(rt)
        μ = intercept + slope_accuracy * condition[i]
        rt[i] ~ Normal(μ, σ)
    end
end


model = model_linear(df.RT; condition=df.Accuracy)
chain_linear = sample(model, NUTS(), 400)

# Summary (95% CI)
quantile(chain_linear; q=[0.025, 0.975])

The effect of Condition is significant, people are on average slower (higher RT) when condition is `"Accuracy"`.
But is our model good?

#### Posterior Predictive Check


In [ ]:
#| output: false

pred = predict(model_linear([(missing) for i in 1:length(df.RT)], condition=df.Accuracy), chain_linear)
pred = Array(pred)

In [ ]:
fig = plot_distribution(df, "Predictions made by Linear Model")
for i in 1:length(chain_linear)
    lines!(Makie.KernelDensity.kde(pred[:, i]), color=ifelse(df.Accuracy[i] == 1, "#388E3C", "#D32F2F"), alpha=0.1)
end
fig

#### The Problem with Linear Models

Reaction time (RTs) have been traditionally modeled using traditional linear models and their derived statistical tests such as *t*-test and ANOVAs. Importantly, linear models - by definition - will try to predict the *mean* of the outcome variable by estimating the "best fitting" *Normal* distribution. In the context of reaction times (RTs), this is not ideal, as RTs typically exhibit a non-normal distribution, skewed towards the left with a long tail towards the right. This means that the parameters of a Normal distribution (mean $\mu$ and standard deviation $\sigma$) are not good descriptors of the data.

![](media/rt_normal.gif)

> Linear models try to find the best fitting Normal distribution for the data. However, for reaction times, even the best fitting Normal distribution (in red) does not capture well the actual data (in grey).

A popular mitigation method to account for the non-normality of RTs is to transform the data, using for instance the popular *log-transform*. 
However, this practice should be avoided as it leads to various issues, including loss of power and distorted results interpretation [@lo2015transform; @schramm2019reaction].
Instead, rather than applying arbitrary data transformation, it would be better to swap the Normal distribution used by the model for a more appropriate one that can better capture the characteristics of a RT distribution.


### Shifted LogNormal Model

One of the obvious candidate alternative to the log-transformation would be to use a model with a Log-transformed Normal distribution.

New parameter, $\tau$ (Tau for delay), which corresponds to the "starting time". We need to set a prior for this parameter, which is usually truncated between 0 and the minimum RT of the data (the logic being that the minimum delay for response must be lower than the faster response actually observed).


In [ ]:
xaxis = range(0, 1, 1000)
lines(xaxis, pdf.(Gamma(1.1, 11), xaxis))

#### Model


In [ ]:
#| code-fold: false

@model function model_lognormal(rt; min_rt=minimum(df.RT), condition=nothing)

    # Priors 
    σ ~ truncated(Normal(0, 0.5); lower=0)
    τ ~ truncated(Gamma(1.1, 11); upper=min_rt)

    intercept ~ Normal(0, 2)
    slope_accuracy ~ Normal(0, 0.5)

    for i in 1:length(rt)
        μ = intercept + slope_accuracy * condition[i]
        rt[i] ~ ShiftedLogNormal(μ, σ, τ)
    end
end

model = model_lognormal(df.RT; condition=df.Accuracy)
chain_lognormal = sample(model, NUTS(), 400)

# Summary (95% CI)
quantile(chain_lognormal; q=[0.025, 0.975])

In [ ]:
#| output: false

# pred = predict(model_lognormal([(missing) for i in 1:length(df.RT)]; condition=df.Accuracy), chain_lognormal)
# pred = Array(pred)

In [ ]:
# fig = plot_distribution(df, "Predictions made by Shifted LogNormal Model")
# for i in 1:length(chain_lognormal)
#     lines!(Makie.KernelDensity.kde(pred[:, i]), color=ifelse(df.Accuracy[i] == 1, "#388E3C", "#D32F2F"), alpha=0.1)
# end
# fig

#### More Conditional Parameters



### ExGaussian Model


- [**Ex-Gaussian models in R: A Tutorial**](https://dominiquemakowski.github.io/easyRT/articles/exgaussian.html)

### Wald Model

Moe from statistical models that *describe* to models that *generate* RT-like data.

## Generative Models (DDM)

Use DDM as a case study to introduce generative models

- [**Drift Diffusion Model (DDM) in R: A Tutorial**](https://dominiquemakowski.github.io/easyRT/articles/ddm.html)

## Other Models (LBA, LNR)


## Additional Resources

- [**Lindelov's overview of RT models**](https://lindeloev.github.io/shiny-rt/): An absolute must-read.
- [**De Boeck & Jeon (2019)**](https://www.frontiersin.org/articles/10.3389/fpsyg.2019.00102/full): A paper providing an overview of RT models.
- [https://github.com/vasishth/bayescogsci](https://github.com/vasishth/bayescogsci)